In [1]:
import numpy as np
import torch
from models import *
from main import *
from configs import cfg
from nltk.translate import bleu_score as bs
import time

In [2]:
cfg['gen_temp'] = 0.4
fname = 'reviews/reviews_tau_0.01.txt'

In [3]:
test_data = np.load('/datasets/home/32/032/sugao/CSE190/pa4/test.npy')

In [4]:
df = load_data('/datasets/cs190f-public/Beeradvocate_TestOriginal.csv')
df.dropna(subset=['review/overall', 'beer/style'],how='any',inplace = True)

In [5]:
N = 3000
start = len(df) % N
cfg['batch_size'] = start
model = load_model(BeerLstmMind, 'models2/e6b6700.pt', cfg)

with torch.no_grad():
    all_txt = generate(model, test_data[0:start], cfg)
    li = tensor2strlist(all_txt)
    save_to_file(li, fname)

cfg['batch_size'] = N
model = load_model(BeerLstmMind, 'models2/e6b6700.pt', cfg)
scores = []
for i_batch in range(start, len(test_data), N):
    with torch.no_grad():
        all_txt = generate(model, test_data[i_batch:i_batch + N], cfg)
        li = tensor2strlist(all_txt)        
        score = calc_score(df, [x for x in range(i_batch, i_batch + 10)], li[0:10])
        scores.append(score)
        save_to_file(li, fname)
        print(i_batch, score)

/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


2323 0.35014149298400593
5323 0.3569402682808938


/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


8323 0.049190657415415534
11323 0.3327264426355143
14323 0.3445337429661781
17323 0.3423328016153426
20323 0.3560839540547193
23323 0.05116451188206174
26323 0.35732247277771156
29323 0.04318696603024291
32323 0.04534597509287905
35323 0.05598889346906769
38323 0.05652028595774142
41323 0.2891223476078993
44323 0.37026513147924517
47323 0.058389421830886384
50323 0.05471650713289312
53323 0.05954139367282448
56323 0.3430342691088023
59323 0.3471104658966625
62323 0.05140052849373915
65323 0.0627311745502811
68323 0.3491210779474666
71323 0.06224602640626996
74323 0.046087405779838025
77323 0.3320956431498245
80323 0.04928484530419629
83323 0.3427749977597686
86323 0.04902640061398098
89323 0.052234661804056205
92323 0.33857776844093024
95323 0.051859268362104236
98323 0.35910739460933266
101323 0.06275020421729878
104323 0.32132194483628757
107323 0.05744890569988377
110323 0.0628265298466463
113323 0.3380571025689078
116323 0.05236586512671986
119323 0.34449697505113935
122323 0.36802

In [ ]:
# lstm-0.4 score median: 0.357
# lstm-10 score median: 0
# lstm-0.01 score median: 0.06

In [6]:
np.median(scores)

0.061846082200194004

In [3]:
training_data = np.load('/datasets/home/32/032/sugao/CSE190/pa4/train.npy')

In [4]:
valid_indices = np.load('./val2.npy')

In [5]:
df = load_data('/datasets/cs190f-public/BeerAdvocateDataset/BeerAdvocate_Train.csv')
df.dropna(subset=['review/text'],how='any',inplace = True)

In [7]:
scores = []
n = 10
cfg['batch_size'] = n
model = load_model(BeerLstmMind, 'models2/e6b6700.pt', cfg)
for i_batch in range(0, 1000, n):
    meta = np.empty((n, 2))
    data = training_data[valid_indices[i_batch:i_batch + n]]
    for j in range(n):
        meta[j, 0] = data[j][0]
        meta[j, 1] = data[j][1]
        
    all_txt = generate(model, meta, cfg)
    li = tensor2strlist(all_txt)
    score = calc_score(df, valid_indices[i_batch:i_batch + n], li)
    scores.append(score)
    print(score)

print(np.median(scores))


/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.3532865912153181


/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.056389223433255864
0.361630984956203
0.36425229529277364
0.050643286536365184
0.3589527138782376
0.05489942350642204
0.3756654947913578
0.05719577280666564
0.3718277547244044
0.3530207954726666
0.3676360869459134
0.37461190706849723
0.3692744729379982
0.34308199709261133
0.3614311636650761
0.3588518717819272
0.054064528469959364
0.3609970982339188
0.06685322498291954
0.3594858081586778
0.3563228886010494
0.35363976006120196
0.05394260717474201
0.05644577711975125
0.363699800531504
0.34841842048516436
0.05568017176727743
0.34791594751284466
0.3566325206249314
0.054786606086544
0.334813695420013
0.36675589482044507
0.37126875376323043
0.36984386441868206
0.3671103107900289
0.36458256175774184
0.3779644730092272
0.36146855374095677
0.34944336687845956
0.36178446923218993
0.3654154756905824
0.05454458676725678
0.359531304053806
0.3735942070746957
0.3784605555481003
0.3728052949625663
0.3518129698290836
0.05455734387771074
0.3624516018258833
0.05372058414627142
0.35983445157892197
0.36517